In [ ]:
!pip install xlrd
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
df=pd.read_excel("news.xls")
df.head()

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])

In [ ]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])


In [ ]:
max_len = 15
padded = pad_sequences(sequences, maxlen=max_len, padding='post')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded, df['label'].values, test_size=0.25, random_state=42)


In [ ]:
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=1,
                    batch_size=5,
                    validation_data=(X_test, y_test))

model.save("model/bilstm_model.h5")

import pickle
with open("tokenizer/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open("label_encoder/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test doğruluğu: {accuracy:.2f}")


In [ ]:
import sys
import pickle
import numpy as np
import tensorflow as tf
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel,
    QLineEdit, QPushButton, QMessageBox
)
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model ve yardımcı objeleri yükle
model = tf.keras.models.load_model("model/bilstm_model.h5")

with open("tokenizer/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("label_encoder/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

max_len = 15  # Eğitimde kullanılan sabit değer

class HaberSiniflandirici(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Türkçe Haber Sınıflandırma")
        self.setGeometry(100, 100, 400, 200)
        self.init_ui()

    def init_ui(self):
        self.layout = QVBoxLayout()

        self.label = QLabel("Bir haber cümlesi girin:")
        self.layout.addWidget(self.label)

        self.input_field = QLineEdit()
        self.layout.addWidget(self.input_field)

        self.button = QPushButton("Tahmin Et")
        self.button.clicked.connect(self.tahmin_et)
        self.layout.addWidget(self.button)

        self.result_label = QLabel("")
        self.layout.addWidget(self.result_label)

        self.setLayout(self.layout)

    def tahmin_et(self):
        metin = self.input_field.text().strip()
        if not metin:
            QMessageBox.warning(self, "Uyarı", "Lütfen bir cümle girin.")
            return

        seq = tokenizer.texts_to_sequences([metin])
        padded_seq = pad_sequences(seq, maxlen=max_len, padding='post')
        prediction = model.predict(padded_seq)
        kategori = label_encoder.inverse_transform([np.argmax(prediction)])

        self.result_label.setText(f"Tahmin edilen kategori: {kategori[0]}")

# Uygulama başlat
if __name__ == "__main__":
    app = QApplication(sys.argv)
    pencere = HaberSiniflandirici()
    pencere.show()
    sys.exit(app.exec_())